In [1]:
import findspark 
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sqlprac").getOrCreate()
spark

In [2]:
path = "../data/youtubevideos.csv"

df = spark.read.csv(path,header=True,inferSchema=True)

In [3]:
df.limit(5).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
#create
df.select("channel_title","title","tags","publish_time","views","likes","dislikes","comment_count","thumbnail_link").createOrReplaceTempView("youtube")

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [11]:
#concat
spark.sql("SELECT concat(channel_title,' :: ',title) from youtube").toPandas()

,"concat(channel_title, :: , title)"
0,CaseyNeistat :: WE WANT TO TALK ABOUT OUR MARR...
1,LastWeekTonight :: The Trump Presidency: Last ...
2,Rudy Mancuso :: Racist Superman | Rudy Mancuso...
3,Good Mythical Morning :: Nickelback Lyrics: Re...
4,nigahiga :: I Dare You: GOING BALD!?
...,...
48132,AaronsAnimals :: The Cat Who Caught the Laser
48133,zefrank1 :: True Facts : Ant Mutualism
48134,Brad Mondo :: I GAVE SAFIYA NYGAARD A PERFECT ...
48135,How It Should Have Ended :: How Black Panther ...


In [12]:
##태그의 갯수 구하기
spark.sql("SELECT tags, LENGTH(tags) - length(replace(tags,'|','')) +1 as num_tags from youtube").toPandas()

,tags,num_tags
0,SHANtell martin,1.0
1,"""last week tonight trump presidency""|""last wee...",4.0
2,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",23.0
3,"""rhett and link""|""gmm""|""good mythical morning""...",27.0
4,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",14.0
...,...,...
48132,"""aarons animals""|""aarons""|""animals""|""cat""|""cat...",14.0
48133,[none],1.0
48134,"""I gave safiya nygaard a perfect hair makeover...",24.0
48135,"""Black Panther""|""HISHE""|""Marvel""|""Infinity War...",22.0


In [13]:
df.select("tags",expr("length(tags) - length(replace(tags,'|','')) +1 as tags")).show()

+--------------------+----+
|                tags|tags|
+--------------------+----+
|     SHANtell martin|   1|
|"last week tonigh...|   4|
|"racist superman"...|  23|
|"rhett and link"|...|  27|
|"ryan"|"higa"|"hi...|  14|
|"ijustine"|"week ...|   7|
|"SNL"|"Saturday N...|  42|
|"5 Ice Cream Gadg...|  13|
|"Trailer"|"Hugh J...|  28|
|"vox.com"|"vox"|"...|  20|
|"NFL"|"Football"|...|  49|
|"The Walking Dead...|  40|
|"marshmello"|"blo...|  23|
|"nowthis"|"nowthi...|  25|
|"shopping for new...|  14|
|"Robots"|"Boston ...|   5|
|"pacific rim"|"pa...|  32|
|"TED"|"TED-Ed"|"T...|  24|
|"ultralight"|"air...|  18|
|"SciShow"|"scienc...|  17|
+--------------------+----+
only showing top 20 rows



In [22]:
#count how many video each channel has, using sql
spark.sql("SELECT channel_title,count(*) as number_of_videos from youtube where channel_title !='None' group by channel_title order by 2 desc").limit(3).toPandas()

,channel_title,number_of_videos
0,ESPN,203
1,The Tonight Show Starring Jimmy Fallon,197
2,TheEllenShow,193


In [34]:
#count how many video each channel has, using dataframe
df.filter("channel_title !='None'").groupBy("channel_title").count().orderBy(desc("count")).limit(3).toPandas()

,channel_title,count
0,ESPN,203
1,The Tonight Show Starring Jimmy Fallon,197
2,TheEllenShow,193


In [36]:
spark.sql("SELECT * FROM youtube").limit(3).toPandas()



,channel_title,title,tags,publish_time,views,likes,dislikes,comment_count,thumbnail_link
0,CaseyNeistat,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,2017-11-13T17:13:01.000Z,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg
1,LastWeekTonight,The Trump Presidency: Last Week Tonight with J...,"""last week tonight trump presidency""|""last wee...",2017-11-13T07:30:00.000Z,2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg
2,Rudy Mancuso,"Racist Superman | Rudy Mancuso, King Bach & Le...","""racist superman""|""rudy""|""mancuso""|""king""|""bac...",2017-11-12T19:05:24.000Z,3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg


In [45]:
df = df.withColumn("views",df['views'].cast("Integer")).withColumn("likes",df['likes'].cast("integer")).withColumn("dislikes",df['dislikes'].cast("integer")).withColumn("comment_count",df["comment_count"].cast("integer"))


In [49]:
df = df.withColumn("published_time",expr("substr(publish_time,0,10) as published_time"))

In [57]:
df.groupBy("published_time").agg(sum("views"),sum("likes"),sum("dislikes")).dropna().orderBy(desc("sum(views)")).toPandas()

,published_time,sum(views),sum(likes),sum(dislikes)
0,2018-05-06,4242190277,133724548,6691797
1,2018-04-20,3509867530,89430524,4382390
2,2018-05-18,3049678775,132720111,4108477
3,2018-04-24,2658528121,73716882,2386088
4,2018-05-04,2328813743,45134783,2982509
...,...,...,...,...
308,2016-10-13,2585,7,0
309,2017-10-21,2322,13,0
310,2017-11-04,1827,3,0
311,2017-05-07,1722,17,3
